## Exercise 2: Bias-variance trade-off and resampling techniques 

#### Overview of nature of the problem: 



In [4]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.utils import resample
import common
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm_notebook

In [5]:
#create data: 
np.random.seed(4155)
n = 15
N = 1000
x = np.sort(np.random.uniform(0, 1, N))
y = np.sort(np.random.uniform(0, 1, N))
z = common.FrankeFunction(x, y)



In [3]:
#TODO: Number of bootstraps? 
n_boostraps = 100
MSE_test = np.zeros(n)
MSE_train = np.zeros(n)
polydegree = np.zeros(n)
#for each degree of poly:
X = common.create_X(x, y, n=n)    
X_train, X_test, y_train, y_test = train_test_split(X, z, test_size=0.2)

#reshape for broadcasting in MSE_test and MSE_val  
y_test = np.reshape(y_test, newshape=(y_test.shape[0],1))
#y_train = np.reshape(y_train, newshape=(y_train.shape[0],1))
#TODO: why scale?!?

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)




for degree in tqdm_notebook(range(n), desc = f"Looping through polynomials up to {n} degrees: "):
    model = LinearRegression()
   #model= make_pipeline(PolynomialFeatures(degree=degree), LinearRegression(fit_intercept=False))
    y_pred = np.empty((y_test.shape[0], n_boostraps))
    y_fit = np.empty((y_train.shape[0], n_boostraps))
    
    for i in range(n_boostraps):
        #bootstrap:
        x_, y_ = resample(X_train_scaled, y_train)
        #fit model to x_,y_ sample:
        #print(f"y_.shape : {y_.shape}")
        clf =  model.fit(x_, y_)
        #fit model and predict on test data:
        y_pred[:, i] = clf.predict(X_test_scaled)
        #predict on train data:
        y_fit[:,i] = clf.predict(X_train_scaled)
        

    polydegree[degree] = degree
    MSE_test[degree] = np.mean( np.mean((y_test - y_pred)**2, axis=1, keepdims=True) )
    y_train_ = np.reshape(y_train, newshape=(y_train.shape[0],1))
    MSE_train[degree] = np.mean( np.mean((y_train_ - y_fit)**2, axis=1, keepdims=True) )
    
    
plt.plot(polydegree, MSE_test, label='MSE_test')
plt.plot(polydegree, MSE_train, label='MSE_train')
plt.xlabel("Model complexity / Polynomial Degree")
plt.ylabel("Prediction Error")
plt.legend()
plt.show()

Looping through polynomials up to 15 degrees:   0%|          | 0/15 [00:00<?, ?it/s]